In [1]:
import numpy as np
import pandas as pd

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from string import Template # For generating prompt template

import os
import gc # grabage collector
# we need to install the sentence transformer and use its embedding to read the faiss index
#cp stands for a copy. This command is used to copy files or groups of files or directories. 
# The -r option tells rm to remove directories recursively, and the -f option tells it to force the removal of files and directories that are read-only or do not exist

!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers

#installing faiss package for reading faiss wikipedia index
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# as per wikipedia faiss index https://www.kaggle.com/datasets/jjinho/wikipedia-2023-07-faiss-index
import faiss
from faiss import write_index, read_index


import ctypes
libc = ctypes.CDLL("libc.so.6")

# installing langchain package# We will use langchain recursive splitter
!pip install langchain --no-index --find-links=file:///kaggle/input/llm-pkg/
from langchain.text_splitter import RecursiveCharacterTextSplitter



from tqdm.auto import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Processing ./sentence-transformers
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126125 sha256=7934d2042232a11dcfa26868ebb3966349e3f489620ae6f2ed080191c43018d0
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2
Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
faiss-gpu is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in links: file:///kaggle/input/llm-pkg/


In [2]:
# Reading the csv file
#df_train = pd.read_csv("./train.csv")
df_final = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
df_final.head(5)

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


READING WIKIPEDIA FILES TO FIND CONTEXT****

In [3]:
# PART 1 - Searching Wikipedia Titles

In [4]:
# loadding the wikipedia faiss index. This will be used for searching
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

In [5]:
# Creating index of prompts i.e q to serach for relavnt wikipedia documents
from sentence_transformers import SentenceTransformer
SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 32

model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half() # The model.half() method in PyTorch is used to convert a model to half-precision. This can be useful for reducing the memory footprint of a model, as half-precision numbers use half the memory as single-precision numbers

In [6]:
prompt_embeddings = model.encode(df_final['prompt'].values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy() # detach to remove gradients.
search_score, search_index = sentence_index.search(prompt_embeddings, 1)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:

del sentence_index # deleting as not required. otherwise it will give memory issue
del prompt_embeddings
_ = gc.collect() # garbage collector..frees up memmory

In [8]:
# PART 2 - Fetching relavant text of wikipedia documents

In [9]:
# getting wikipedia documents 
df_wiki = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                     columns=['id', 'file'])

In [10]:
wikipedia_file_data = []

for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx
    _df = df_wiki.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)

  0%|          | 0/200 [00:00<?, ?it/s]

In [11]:
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

In [12]:
wikipedia_file_data.head(5)


,id,prompt_id,file
0,1317,89,a.parquet
1,1398957,63,a.parquet
2,1691321,153,a.parquet
3,311855,162,a.parquet
4,347027,33,a.parquet


In [13]:
del df_wiki
_ = gc.collect()
libc.malloc_trim(0)

1

In [14]:
import os
WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)

wiki_text_data = []

for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

    _df_temp = _df[_df['id'].isin(_id)].copy()
    del _df
    _ = gc.collect()
    libc.malloc_trim(0)
    wiki_text_data.append(_df_temp)
wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

  0%|          | 0/22 [00:00<?, ?it/s]

In [15]:
context_df = wikipedia_file_data.merge(wiki_text_data,on='id')
print(len(context_df))

200


In [16]:
context_df.head()


,id,prompt_id,file,text
0,1317,89,a.parquet,"In modern physics, antimatter is defined as ma..."
1,1398957,63,a.parquet,Atomism or social atomism is a sociological th...
2,1691321,153,a.parquet,The American Petroleum Institute (API) is the ...
3,311855,162,a.parquet,thumb|100px|Spin — an aggravated stall and aut...
4,347027,33,a.parquet,"thumb|Samples of amorphous metal, with millime..."


In [17]:
# Spliting the wiki text in the context df in chunk size

chunk_size = 1000
chunk_overlap = 100

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

split_text =[]
for i in range(len(context_df)):
    split_text.append ( r_splitter.split_text(context_df.loc[i,'text']))
context_df['split'] = split_text


In [18]:
context_df.head()

,id,prompt_id,file,text,split
0,1317,89,a.parquet,"In modern physics, antimatter is defined as ma...","[In modern physics, antimatter is defined as m..."
1,1398957,63,a.parquet,Atomism or social atomism is a sociological th...,[Atomism or social atomism is a sociological t...
2,1691321,153,a.parquet,The American Petroleum Institute (API) is the ...,[The American Petroleum Institute (API) is the...
3,311855,162,a.parquet,thumb|100px|Spin — an aggravated stall and aut...,[thumb|100px|Spin — an aggravated stall and au...
4,347027,33,a.parquet,"thumb|Samples of amorphous metal, with millime...","[thumb|Samples of amorphous metal, with millim..."


PREPARING THE PROMPT

In [19]:
# text  = """
# Give below is a question labelled as 'Q': , context labelled as 'T' :for additional ifnormation and 5 possible answers to the question labelled as 'A':,'B':,'C':,'D':,'E':. 
#     'Answer the following question by outputting the letters A, B, C, D, and E '\
#     'in order of the most likely to be correct to the to least likely to be correct
# 'Q' : $q
# 'T' : $t

# 'A' : $a
# 'B' : $b
# 'C' : $c
# 'D' : $d
# 'E' : $e


# """

# template = Template(text)

In [20]:
# def format_dataframe(df, context_df):
    
#     final_prompt = []
#     for i in range(len(df)):
#         q = df.loc[i,'prompt']
#         a = df.loc[i,'A']
#         b = df.loc[i,'B']
#         c = df.loc[i,'C']
#         d = df.loc[i,'D']
#         e = df.loc[i,'E']
        
#         text = context_df[context_df['prompt_id'] == i].iloc[0]['split']
#         if text != []:
#             text_df = pd.DataFrame(text,columns=['text'])
#             vectors = model.encode(text_df['text'])
#             vector_dimension = vectors.shape[1]
#             index = faiss.IndexFlatL2(vector_dimension)
#             faiss.normalize_L2(vectors)
#             index.add(vectors)


#             search_vector = model.encode(q)
#             _vector = np.array([search_vector])
#             faiss.normalize_L2(_vector)

#             k = 1
#             distances, ann = index.search(_vector, k=k)
#             chunk = text[ann[0,0]]

#         final_prompt.append(template.substitute(q=q,a = a,b=b,c=c,d=d,e=e,t=chunk))
#      #   break
    
#     return final_prompt




In [21]:
def format_dataframe(df, context_df):
    
 #   final_prompt = []
    for i in range(len(df)):
        q = df.loc[i,'prompt']
        a = df.loc[i,'A']
        b = df.loc[i,'B']
        c = df.loc[i,'C']
        d = df.loc[i,'D']
        e = df.loc[i,'E']
        chunk = ''
        
        text = context_df[context_df['prompt_id'] == i].iloc[0]['split']
        if text != []:
            text_df = pd.DataFrame(text,columns=['text'])
            vectors = model.encode(text_df['text'])
            vector_dimension = vectors.shape[1]
            index = faiss.IndexFlatL2(vector_dimension)
            faiss.normalize_L2(vectors)
            index.add(vectors)


            search_vector = model.encode(q)
            _vector = np.array([search_vector])
            faiss.normalize_L2(_vector)

            k = 1
            distances, ann = index.search(_vector, k=k)
            chunk = text[ann[0,0]]

        df.loc[i,'context'] = chunk

  #      final_prompt.append(template.substitute(q=q,a = a,b=b,c=c,d=d,e=e,t=chunk))
     #   break
    
    return df

In [22]:
model_prompt = format_dataframe(df_final,context_df)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
#model_df.head()

************PROMPTING THE MODEL TO GET RESPONSE

In [25]:
model_prompt[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)

In [26]:
from transformers import AutoTokenizer,AutoModelForMultipleChoice

# The path of the model checkpoint we want to use
model_dir = '/kaggle/input/huggingface-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at /kaggle/input/huggingface-bert/bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [27]:
test_df = pd.read_csv("test_context.csv")
test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df['context'] = test_df['context'].apply(lambda x: str(x))
test_df["prompt"] = test_df["context"].apply(lambda x: x[:2500]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'B'

In [28]:
test_df

,prompt,context,A,B,C,D,E,id,answer
0,of the source is distorted. The mass of the da...,of the source is distorted. The mass of the da...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,0,B
1,Dynamic scaling (sometimes known as Family-Vic...,Dynamic scaling (sometimes known as Family-Vic...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,1,B
2,"center.Matthews, Jeff (2005) Symbols of Naples...","center.Matthews, Jeff (2005) Symbols of Naples...",The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,2,B
3,"a negative metric or wrong statistics, or disc...","a negative metric or wrong statistics, or disc...",Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,3,B
4,File:Diffraction sharp edge.gif|Diffraction on...,File:Diffraction sharp edge.gif|Diffraction on...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,4,B
...,...,...,...,...,...,...,...,...,...
195,Second Theorem The three moment equation expre...,Second Theorem The three moment equation expre...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,195,B
196,slowing or speeding of an internet service by ...,slowing or speeding of an internet service by ...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,196,B
197,Recalescence is an increase in temperature tha...,Recalescence is an increase in temperature tha...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",197,B
198,readers it can appear that some dynamical quan...,readers it can appear that some dynamical quan...,Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,198,B


## TRAINING THE MODEL

In [29]:
# For convenience we'll turn our pandas Dataframe into a Dataset
from datasets import Dataset
from transformers import AutoTokenizer

In [30]:
train_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')

In [31]:
train_ds = Dataset.from_pandas(train_df)

In [32]:
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example

In [33]:
# Following datacollator (adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice)
# will dynamically pad our questions at batch-time so we don't have to make every question the length
# of our longest question.
from datasets import Dataset
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch


In [34]:
from torch.utils.data import DataLoader
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)

  0%|          | 0/200 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [47]:
model_ckpts = {
    "/kaggle/input/llm-science-run-context-2": 1,
#     "/kaggle/input/how-to-train-open-book-model-part-1/model_v2": 0.35,
#     "/kaggle/input/2023kagglellm-deberta-v3-large-model1": 0.1,
#     "/kaggle/input/my-1-epoch": 0.1,
#     "/kaggle/input/llm-se-debertav3-large": 0.08,
#     "/kaggle/input/science-exam-trained-model-weights/run_0": 0.06,
#     "/kaggle/input/science-exam-trained-model-weights/run_2": 0.06
}

In [48]:
preds = None
for ckpt in tqdm(model_ckpts.keys()):
    print(ckpt + ':' + str(model_ckpts[ckpt]))
    model = AutoModelForMultipleChoice.from_pretrained(ckpt).cuda()
    model.eval()
    
    test_predictions = []
    for batch in tqdm(test_dataloader):
        for k in batch.keys():
            batch[k] = batch[k].cuda()
        with torch.no_grad():
            outputs = model(**batch)
        test_predictions.append(outputs.logits.cpu().detach())
    predictions = torch.cat(test_predictions)
    print(predictions.shape)
    if preds is not None:
        preds += predictions * model_ckpts[ckpt]
    else:
        preds = predictions * model_ckpts[ckpt]
    del model

predictions_as_ids = np.argsort(-preds, 1)

predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
# predictions_as_answer_letters[:3]

predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

  0%|          | 0/1 [00:00<?, ?it/s]

/kaggle/input/llm-science-run-context-2:1


  0%|          | 0/200 [00:00<?, ?it/s]

torch.Size([200, 5])


In [50]:
submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

In [ ]:
# # Now we'll instatiate the model that we'll finetune on our public dataset, then use to
# # make prediction on the private dataset.
# from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
# model = AutoModelForMultipleChoice.from_pretrained(model_dir)

In [ ]:
# # The arguments here are selected to run quickly; feel free to play with them.
# model_dir = 'finetuned_bert'
# training_args = TrainingArguments(
#     output_dir=model_dir,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     learning_rate=5e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     report_to='none'
# )

In [ ]:
# # Generally it's a bad idea to validate on your training set, but because our training set
# # for this problem is so small we're going to train on all our data.
# tokenized_train_ds = train_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
# trainer = Trainer(
#     model=model,
#     args=training_args,
# #     train_daataset=tokenized_train_ds,
#     eval_dataset=tokenized_train_ds,
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
# )

In [ ]:
# # Training should take about a minute
# trainer.train()

In [ ]:
# # The following function gets the indices of the highest scoring answers for each row
# # and converts them back to our answer format (A, B, C, D, E)
# import numpy as np
# def predictions_to_map_output(predictions):
#     sorted_answer_indices = np.argsort(-predictions)
#     top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
#     top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
#     return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [ ]:
# # Let's double check our output looks correct:
# predictions_to_map_output(predictions.predictions)

In [ ]:
# # Now we can load up our test set to use our model on!
# # The public test.csv isn't the real dataset (it's actually just a copy of train.csv without the answer column)
# # but it has the same format as the real test set, so using it is a good way to ensure our code will work when we submit.
# test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
# test_df.head()

In [ ]:
# # Here we'll generate our "real" predictions on the test set
# test_predictions = trainer.predict(tokenized_test_dataset)

In [ ]:
# # Now we can create our submission using the id column from test.csv
# submission_df = test_df[['id']]
# submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)

# submission_df.head()

In [ ]:
# # Once we write our submission file we're good to submit!
# submission_df.to_csv('submission.csv', index=False)

**SUBMISSION **

In [102]:
# submission = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/sample_submission.csv', index_col='id')

# i = 0
# for text in model_prompt:
#     inputs = tokenizer_llm(text, return_tensors="pt").to(device)
#     outputs = model_llm.generate(**inputs)
#     answer = tokenizer_llm.batch_decode(outputs, skip_special_tokens=True)
#     submission.loc[i,'prediction'] = post_process(answer)
#     i = i+1

# submission.to_csv('submission.csv')

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
